In [1]:
import torch
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt

epochs = 50

In [2]:
def plot(true_values, predicted_values):
    """Plot true vs predicted counts and loss."""
   

    fig = plt.figure()
    plt.scatter(true_values, predicted_values)

    plt.title('Train')
    plt.xlabel('True value')
    plt.ylabel('Predicted value')
    plt.show()

In [3]:
from base import gpu_test

device = gpu_test.get_device()
device.type

Using device: cuda

GeForce RTX 3070
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


'cuda'

In [4]:
from lbt.search.backbone import model_search
from lbt import architect
from base import FCRNA
from lbt.architect import cusloss

criterion = torch.nn.MSELoss()
criterion = criterion.cuda()

criterion_stud = torch.nn.MSELoss()
criterion_stud = criterion_stud.cuda()


model = model_search.Network(input_c = 3, c = 24, num_classes = 1, criterion = criterion, depth = 3, device = device)
model = model.cuda()

student  = FCRNA.FCRN_A(criterion_stud, input_filters=3, filters=64, N=2)
student = student.cuda()

optimizer = torch.optim.SGD(model.parameters(), 0.025, momentum=0.9, weight_decay=3e-4)
optimizer_stud =  torch.optim.SGD(student.parameters(), 0.025, momentum=0.9, weight_decay=3e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, float(epochs))

architect = architect.Architect(model, student)

C:\Users\hicks\anaconda3\lib\site-packages\torch\optim\adam.py:48: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


In [5]:
from base import data_loader
from base import train
from torch.autograd import Variable


data_loader = train.get_loader('cell', batch_size=1)
data_loader_unlabeled = train.get_loader('cell', batch_size=1)

train_queue = data_loader['train']
valid_queue = data_loader['valid']
unlabeled_queue = data_loader_unlabeled['train']

In [6]:
def train(train_queue, valid_queue, unlabeled_queue, model, student, architect, criterion, criterion_stud, optimizer, optimizer_stud, lr):
    
    true_values = []
    predicted_values = []
    
    for image, target in tqdm(train_queue):
        model.train()

        image = Variable(image, requires_grad=False).cuda()
        target = Variable(target, requires_grad=False).cuda()

        # get a random minibatch from the search queue with replacement
        input_search, target_search = next(iter(valid_queue))
        input_search = Variable(input_search, requires_grad=True).cuda()
        target_search = Variable(target_search, requires_grad=True).cuda()

        # get a random minibatch from the cifar-100 queue with replacement
        input_unlabeled, target_unlabeled = next(iter(unlabeled_queue))
        input_unlabeled = Variable(input_unlabeled, requires_grad=True).cuda()
        target_unlabeled = Variable(target_unlabeled, requires_grad=True).cuda()

        architect.step(image, target, input_search, target_search, input_unlabeled, lr, optimizer, unrolled=True)
        architect.step1(image, target, input_search, target_search, input_unlabeled, lr, optimizer, optimizer_stud, unrolled=True)

        optimizer.zero_grad()
        logits = model(image)
        loss = criterion(logits, target)

        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), 5)
        optimizer.step()

        optimizer_stud.zero_grad()
        l1 = model(input_unlabeled)
        logits1 = student(input_unlabeled)
        loss1 = cusloss(logits1, l1.detach())


        loss1.backward()
        #nn.utils.clip_grad_norm(student.parameters(), args.grad_clip)
        optimizer_stud.step()

        optimizer_stud.zero_grad()
        logits2 = student(image)
        loss2 = criterion_stud(logits2, target)

        loss2.backward()
        #nn.utils.clip_grad_norm(student.parameters(), args.grad_clip)
        optimizer_stud.step()
        
        for true, predicted in zip(label, logits):

            true_counts = torch.sum(true).item() / 100
            predicted_counts = torch.sum(predicted).item() / 100

            # update current epoch results
            true_values.append(true_counts)
            predicted_values.append(predicted_counts)
        
        print(loss.item())
        plot(true_values, predicted_values)
        
        

In [7]:
for epoch in range(epochs):
    print(epoch)
    
    scheduler.step()
    lr = scheduler.get_lr()[0]
    
    train(train_queue, valid_queue, unlabeled_queue, model, student, architect, criterion, criterion_stud, optimizer, optimizer_stud, lr)

0


C:\Users\hicks\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
C:\Users\hicks\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:508: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [ ]:
for name, param in student.named_parameters():
    if param.requires_grad:
        print(name, param.data)